<h2>Working with Semi-Structured Data: Python Dictionaries, JSON, MongoDB and PyMongo</h2>

<h3>Introduction</h3>

Much of the data I work with comes from online sources. Typically, the data is found within the confines of the HTML code underlying one or more web pages, (eventually) involves analysis at the <i>meso</i> level (100-10K records), and on the surface appears on the surface to be well-structured or, if not, at least amenable to being well structured. By "structured" I mean it's the type of data you encounter in the tabular world of SQL databases and spreadsheets.  Here, the rows of the table usually represent the objects or entities of interest, the columns are parameters or variables associated with those objects or entities, and each cell contains a (single) value of the parameter or variable for a particular entity or object. In Python, this sort of data is easily represented with a <i>dataframe</i>, easily analyzed with the <i>pandas</i> package, and easily stored in a comma or tab delimited file, a spreadsheet file, or a SQL database.

Take, for example, the data I used in analyzing the best selling songs (by volume) on Billboards Top 100 chart over the last 60 years (see: <a href="https://github.com/daveking63/Billboard-and-RIAA-datasets/blob/master/BB-Top600_Songs.txt">the Top 600 Songs text file in the Billboard & RIAA repository</a>). On the surface, this is what data looks like:
    
![Top Selling Billboard Song](chubbyCheckerTwist2.jpg "Sample song from Billboard Top Selling Songs")

This is the associated HTML code for this entry:    

![Underlying HTML Code](chubbyCheckerHTMLCode.jpg "Sample song underlying HTML Code")

This is what the entry looked like in an Excel spreadsheet I created to store this and the other entries on the page:

![Spreadsheet Row](chubbyCheckerExcel.jpg "Sample song entry in spreadsheet")

In terms of extracting, formatting, and storing Web data, this is about as easy as it gets, but even here you encounter issues with the tabular/dataframe representation. Consider this question: On how many of the best selling songs did Paul McCartney sing? Here's the answer:

![McCartney Songs](paulMcCartneySongs.jpg "Songs in which McCartney participated")

For the Top 100 he sang on 4 of the songs. For the Top 600, he sang on 9 songs

Now think about this? How did I get this answer? A simple query like "show me the rows where the column/field 'Artist' equals 'Paul McCartney'" won't work.  With this query, the answer is 0 because there is no row (i.e. song) where Paul McCartney sang solo. Even if the query were "show me the rows where the 'Artist' contains (the substring) 'Paul McCartney'", it still wouldn't work because it would ignore those songs sung by the 'The Beatles.' Answering a question of this sort obviously requires more information and a different representation of the 'Artist' field beyond what's provided by a simple string. Here, an often used 'klug' for handling it with a tabular format is to add another column for duos and groups containing a delimited string with the names of all the individual artists involved (e.g. 'Paul McCartney|John Lennon|George Harrison|Ringo Starr')  

\[As an aside, the <i>pandas</i> package provides a number of methods/functions (like 'contains') for dealing with strings that are equivalent to Python's general 'str' and 'regex' functions.\]

Moving from questions about the 'Artist' column, let's look at the 'SongTitle' column. How many songs contain the word 'LOVE' in the SongTitle? Pretty straightforward query, "Show me the rows where the 'SongTitle' contains the substring 'LOVE.' How about this one: "Which word occurs most frequently in the titles?" Here's the answer:

<TABLE>
<TR><TH>Rank</TH><TH>Word</TH><TH>Freq</TH></TR>
<TR><TD>0</TD><TD>THE</TD><TD>76</TD></TR>
<TR><TD>1</TD><TD>YOU</TD><TD>75</TD></TR>
<TR><TD>2</TD><TD>LOVE</TD><TD>50</TD></TR>
<TR><TD>3</TD><TD>ME</TD><TD>41</TD></TR>
<TR><TD>4</TD><TD>TO</TD><TD>37</TD></TR>
<TR><TD>5</TD><TD>IT</TD><TD>31</TD></TR>
<TR><TD>6</TD><TD>OF</TD><TD>26</TD></TR>
<TR><TD>7</TD><TD>MY</TD><TD>26</TD></TR>
<TR><TD>8</TD><TD>IN</TD><TD>25</TD></TR>
<TR><TD>9</TD><TD>THAT</TD><TD>16</TD></TR>
</TABLE>

This is a list of the 10 most frequently occurring words. Number 1 is 'YOU', 2 is 'I', and so on. Pretty boring. Actually, they're basically English stopwords except for number 4 which is the word 'LOVE.' Anytime you analyze any sort of English corpus, the stopwords usually rule. How did I create this frequency distribution? You can't do it with 'substring' functions (at least not sensibly). In Python it's best done with a different sort of data structure along with some rudimentary text analysis. If the question had been: which 'SongTitle's deal with 'love'? The analysis is no longer rudimentary.

For me, the moral of this example is that even simple data analysis can move out of the realm of the 'structured' tabular world pretty quickly, especially with online data which often deals with evolving schemas, multiple entities, nested hierarchies, lists of values (like tags), text of all shapes and sizes, links, images, video and the like.  These things are often treated as <i>unstructured</i> or <i>semi-structured</i> data. Wikipedia provides a pretty good summary of both terms:

<ol>
    <li><i>Semi-Structured Data</i> -- "... a form of structured data that does not conform with the formal structure of data models associated with relational databases or other forms of data tables, but nonetheless contains tags or other markers to separate semantic elements and enforce hierarchies of records and fields within the data."</li>
    <li><i>Unstructured Data</i> -- "...is data that either does not have a pre-defined data model or is not organized in a pre-defined manner. Unstructured data is typically text-heavy, but may contain data such as dates, numbers, and facts as well. This results in irregularities and ambiguities that make it difficult to understand using traditional programs as compared to data stored in fielded form in databases or annotated (semantically tagged) in documents." In this discussion, raw images and video also fall into this category.</li>
</ol>

For the rest of this notebook, I'm going focus the discussion on the use of a combination of Python <i>dictionaries</i>, the "lightweight data-interchange format" <i>JSON</i>, the NoSQL document database <i>MongoDB</i> for representing, storing, and analyzing 'semi-structured' data, and the Python library/driver <i>PyMongo</i> used to access MongoDB's capabilities from Python programs. 
    
I've limited the discussion to 'semi-structured' data versus 'unstructured' data for a variety of reasons. First, in analyzing 'unstructured' data, the general idea is usually to convert it into either a semi-structured or structured form so that subsequent exploration and analysis can be more easily performed. This is why we often convert unstructured text into a document-term matrix (e.g. 'bag of n-grams') or an image into matrix of pixels, so we can apply well known procedures to the matrix and it's values. Second, I've covered text analysis and natural language processing in detail in some of my other repositories. Finally, as other authors have suggested, the complexities of image and video analysis is beyond the scope of this discussion and requires a dissertation of it's own (which I'll probably provide in a subsequent project and repository).

<h3><a class="anchor" id="toc">Table of Contents</a></h3>

1. [Python Dictionaries](#dictionaries)<br>
2. [JSON](#JSON)<br>
3. [MongoDB](#mongo)


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

<a class="anchor" id="dictionaries"></a>
<h3>Representing and Creating Data Sets with Lists and Dictionaries</h3>
<a href='#toc'>Return to TOC</a>

There are an extensive number of general articles discussing the structure and uses of Python's base collections including: lists, tuples, sets and dictionaries. It's not uncommon to find all of them being used for various purposes in the same Python program.  However, in this part of the discussion the emphasis is on representing and creating datasets that can be used for analytical purposes.  For these purposes, lists and dictionaries come to the forefront. 

<a class="anchor" id="lists"></a>
<h4>Lists Defined</h4>
<a href='#toc'>Return to TOC</a>

The datasets created with lists share a lot in common with arrays, tables and dataframes. Consider, for example, the following Python lists used to represent a few of the songs in Billboard data discussed above:

In [3]:
song1 = [1,'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s']
song2 = [2,'Santana Featuring Rob Thomas', 'SMOOTH', 'Duo/Group', 'Rock', '90s']
song3 = [3,'Bob Darin', 'MACK THE KNIFE', 'Male', 'Pop', '50s']
song4 = [4,'Mark Ronson Featuring Bruno Mars', 'UPTOWN Funk!', 'Duo/Group', ' Pop', '10s']

print('Sample Lists:')
print('song1:', song1)
print('song2:', song2)
print('song3:', song3)
print('song4:', song4)

Sample Lists:
song1: [1, 'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s']
song2: [2, 'Santana Featuring Rob Thomas', 'SMOOTH', 'Duo/Group', 'Rock', '90s']
song3: [3, 'Bob Darin', 'MACK THE KNIFE', 'Male', 'Pop', '50s']
song4: [4, 'Mark Ronson Featuring Bruno Mars', 'UPTOWN Funk!', 'Duo/Group', ' Pop', '10s']


A list is a 'mutable' collection of values separated by 'commas' and enclosed within brackets '[]'. The values can be numbers, strings, and other lists.  The values are ordered and indexed by position, they're changeable or mutable, and duplicates are allowed. The fact that a list is mutable means that it can be changed after it is created. Some of the key changes that can be made include: append, clear, copy, extend, insert, pop, remove, reverse, and sort (e.g. see <a href= https://docs.python.org/3/tutorial/datastructures.html#>the Python Tutorial on Lists provided by 'python.org'</a>).

<h4>List of Lists: Emulating an Array, Table or Dataframe</h4>

Just like a numpy array or a pandas dataframe, values in the list can be accessed or selected by their position or location in the list. As usual, the positions start at 0 and end at N-1 where N is the length of the list. So, for instance, we can see that the artist (name) is in position 1 (rank is at position 0) which means we can assess it by:

    artistName = songID[position]

In [4]:
artistName = song1[1]
print('Top Ranked Artist:', artistName)
artistName = song4[1]
print('4th Ranked Artist:', artistName)

Top Ranked Artist: Chubby Checker
4th Ranked Artist: Mark Ronson Featuring Bruno Mars


If we wanted to know the names of the artists for all 4 of the songs in the sample of lists, we could create a list of the list names (variables) and then use a 'for loop' to iterate the values:

In [5]:
songs = [song1, song2, song3, song4]
for i in range(0,4):
    print('Artist of Song Ranked', i + 1, 'is', songs[i][1])

Artist of Song Ranked 1 is Chubby Checker
Artist of Song Ranked 2 is Santana Featuring Rob Thomas
Artist of Song Ranked 3 is Bob Darin
Artist of Song Ranked 4 is Mark Ronson Featuring Bruno Mars


In [6]:
# or slightly more succinctly
for song in songs:
    print('Artist of Song Ranked', song[0], 'is', song[1])

Artist of Song Ranked 1 is Chubby Checker
Artist of Song Ranked 2 is Santana Featuring Rob Thomas
Artist of Song Ranked 3 is Bob Darin
Artist of Song Ranked 4 is Mark Ronson Featuring Bruno Mars


Of course, these simple selections assume that: (1) the positions of the fields remain the same from one list to the next; and (2) we remember which field is in which position. Assuming that the former is true, we can assist our memories by creating variables to be used in place of the position numbers and using the range function to assign the position number to the corresponding variable.

In [7]:
range(6)
# there are 6 fields ranging from 0 to 5
    
Rank, Artist, SongTitle, Gender, Genre, Decade = range(6)
#This assigns value of 0 to Rank, 1 to Artist, ..., 5 to Decade

for song in songs:
    print('Artist of Song Ranked', song[Rank], 'is', song[Artist])

range(0, 6)

Artist of Song Ranked 1 is Chubby Checker
Artist of Song Ranked 2 is Santana Featuring Rob Thomas
Artist of Song Ranked 3 is Bob Darin
Artist of Song Ranked 4 is Mark Ronson Featuring Bruno Mars


Finally, we could eliminate the separate variables representing each of the individual songs (e.g. song1, song2, etc.) and simply create a list of lists to represent the data. In this way, we can access the values in all the lists in much the same way we access the values in a 2 dimensional array.

In [8]:
songs = [[1,'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s'],
            [2,'Santana Featuring Rob Thomas', 'SMOOTH', 'Duo/Group', 'Rock', '90s'],
            [3,'Bob Darin', 'MACK THE KNIFE', 'Male', 'Pop', '50s'],
            [4,'Mark Ronson Featuring Bruno Mars', 'UPTOWN Funk!', 'Duo/Group', ' Pop', '10s']]

print('Accessing Artist for Song at Rank 2:', songs[1][1])
print("")

print('All Artists in Sample:')
for i in range(0,len(songs)):
    print('Artist of Song Ranked', songs[i][0], 'is', songs[i][1])

Accessing Artist for Song at Rank 2: Santana Featuring Rob Thomas

All Artists in Sample:
Artist of Song Ranked 1 is Chubby Checker
Artist of Song Ranked 2 is Santana Featuring Rob Thomas
Artist of Song Ranked 3 is Bob Darin
Artist of Song Ranked 4 is Mark Ronson Featuring Bruno Mars


So, we're on our way to producing a data structure that crudely represents a dataframe (without many of the bells and whistles). We might as well use a dataframe instead. Even with a dataframe, we still encounter hurdles, especially when you expand your horizon to other sorts of analysis. As noted at the top of this notebook: what if you want to represent and analyze individual performers within the duos or groups (e.g. how many songs in Top 600 did she, he or they sing? what was the average ranking of these songs? how did the ranks for an individual singer change across time? etc.).  Or, what if you want to analyze the lyrics associated with these songs? what sort of representaton would you need to incorporate the lyrics? It is possible to address many of these sorts of questions usings lists, but they are aren't handled well nor robustly by tabular structures of this sort.  It's situations like these where Python dictionaries can come in handy.

<a class="anchor" id="dictionaries"></a>
<h3>Dictionaries Defined</h3>
<a href='#toc'>Return to TOC</a>

Like a list, a python <i>dictionary</i> (denoted 'dict') is also a 'mutable' collection, but this time the collection consists of 'key:value' pairs where:

<ol>
    <li>each 'key' is an unordered, immutable data type (string, number of tuples) that is 'hashable' and unique with the dictionary (a point that will be clarified when we look at nested dictionaries). If the key is a string it must be enclosed in quotes. The fact that a key is a unique means that we can use the key to index and rapidly access any pair.</li>
    <li>each 'value' is mutable and can be of any data type including another dictionary (dict). Again, if the value is a string, it must be enclosed in quotes.</li>
    <li>the key and value within a pair are separated by a colon</li>
    <li>the pairs within a dictionary are separated by commas</li>
    <li>the collection of pairs is encased in curly braces '{}'.</li>
</ol>

For the Billboard data used above this is what the dictionary for a single song might look like: 

In [9]:
song1D = {'rank': 1, 'artist':'Chubby Checker', 'songTitle':'The Twist', 'gender':'Male', 'genre':'R&B','decade':'60s'}
print('type of song1D', type(song1D))
print('song1D:', song1D)

type of song1D <class 'dict'>
song1D: {'rank': 1, 'artist': 'Chubby Checker', 'songTitle': 'The Twist', 'gender': 'Male', 'genre': 'R&B', 'decade': '60s'}


This is pretty straight forward. 'song1D' is the name of this dict. It has 6 'key:value' pairs with each (unique) key representing a field name and values representing the field values for this particular song. Here, all of the keys are strings (enclosed in quotes). Similarly, most of the values are strings except for 'rank' which is a number.

Another way to create a dictionary is with the 'dict' function. The 'dict' function, which is illustrated below, has a couple of options:

<ol>
    <li>dict(key1=value1,key2=value2,...keyN=valueN)</li>
    <li>dict([(key1,value1),(key2,value2),...(keyN,valueN)])</li>
<ol>

In [10]:
print("Using the 'dict' function to create a dictionary - Option 1: key=value syntax")
print("e.g. dict(rank=1, artist='Chubby Checker'...)")
print("")
song1D = dict(rank=1,artist='Chubby Checker',songTitle='The Twist',gender='Male',genre='R&B',decade='60s')
print('song1D:', song1D)
print("")
print("Using the 'dict' function to create a dictionary - Option 2: (key,value) syntax")
print("e.g.  dict([('rank',1),('artist','Chubby Checker')...]")
print("")
song1D = dict([('rank',1),('artist','Chubby Checker'),('songTitle','The Twist'),('gender','Male'),('genre','R&B'),('decade','60s')])
print('song1D:', song1D)

Using the 'dict' function to create a dictionary - Option 1: key=value syntax
e.g. dict(rank=1, artist='Chubby Checker'...)

song1D: {'rank': 1, 'artist': 'Chubby Checker', 'songTitle': 'The Twist', 'gender': 'Male', 'genre': 'R&B', 'decade': '60s'}

Using the 'dict' function to create a dictionary - Option 2: (key,value) syntax
e.g.  dict([('rank',1),('artist','Chubby Checker')...]

song1D: {'rank': 1, 'artist': 'Chubby Checker', 'songTitle': 'The Twist', 'gender': 'Male', 'genre': 'R&B', 'decade': '60s'}


Imagine having to type in dictionaries for all 600 songs using one of these 3 methods. Obviously, it would be an extremely tedious task in part because you have to repeat the same set of keys over and over and over. Given this syntax, it would even be tedious if you wrote a program to do it. Fortunately, there are a couple of other alternatives, which are also verbose, but which can be programmed in a more straightforward fashion.

The first of these involves entering each 'key:value' pair one at a time. The general syntax is: 

    dictName[key] = value

In order to use this option, you first have to create an empty dictionary (i.e. dictName = {}), then add the pairs. 

In [11]:
print("Entering each 'key:value' pair one at a time")
print("e.g. song1D['Rank'] = 1")

song1D = {}  #start with empty dictionary then add key:value pairs
song1D['Rank'] = 1
song1D['Artist'] = 'Chubby Checker'
song1D['Title'] = 'THE TWIST'
song1D['Gender'] = 'Male'
song1D['Genre'] = 'R&B'
song1D['Decade'] = '60s'
print("")
print("song1D:", song1D)

Entering each 'key:value' pair one at a time
e.g. song1D['Rank'] = 1

song1D: {'Rank': 1, 'Artist': 'Chubby Checker', 'Title': 'THE TWIST', 'Gender': 'Male', 'Genre': 'R&B', 'Decade': '60s'}


The second option involves using a combination of the 'zip' function with the 'dict' function. In this case, the 'zip' function enables us to take two lists of equal length and create a collection of tuples, pairing members in the first list with those members in the second list occupying the same positions (i.e. 0 with 0, 1 with 1, ...). Obviously, this assumes that the keys are in one list and the values in a second.

In [12]:
print("Using the zip function to create a dictionary")
print("")
print("Create a list of keys, e.g. keyList")
keyList = ['Rank','Artist','Title','Gender','Genre','Decade']
print("keyList:", keyList)
print("")
print("Create a list of values, e.g. song1")
song1 = [1,'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s']
print("song1:", song1)
print("")
print("zip keyList with song1 list to produce dictionary: dict(zip(keyList,song1))")
song1D = dict(zip(keyList,song1D))
print("song1Dict:", song1D)

Using the zip function to create a dictionary

Create a list of keys, e.g. keyList
keyList: ['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade']

Create a list of values, e.g. song1
song1: [1, 'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s']

zip keyList with song1 list to produce dictionary: dict(zip(keyList,song1))
song1Dict: {'Rank': 'Rank', 'Artist': 'Artist', 'Title': 'Title', 'Gender': 'Gender', 'Genre': 'Genre', 'Decade': 'Decade'}


Note: Sans the 'dict' function, the 'zip' function produces a zip object containing the combined tuples. One way to see the tuples is to convert the zip object to a list.

In [13]:
print("Displaying zip object as a list: list(zip(keyList,song1))")
print(list(zip(keyList,song1)))

Displaying zip object as a list: list(zip(keyList,song1))
[('Rank', 1), ('Artist', 'Chubby Checker'), ('Title', 'THE TWIST'), ('Gender', 'Male'), ('Genre', 'R&B'), ('Decade', '60s')]


So, given these methods, let's look at how to create dictionaries for a list of songs. In order to illustrate the process, I'm going to 'fudge' a little and start with the keyList and the list of (4) songs created earlier. 

The first method is based on the option of entering the pairs one at a time. The second approach uses the 'zip' option. In both instances the basic idea is to create an overall dictionary (called 'songsDict') to hold the dictionaries for each of the individual songs. Since 'songsDict' is a dictionary it will be made up of 'key:value' pairs. Here, each of the keys will be a label or index that delineates an individual song and that can be used to access that song's dictionary. The question is: what form should these keys take. Any number of forms can be used but more than likely we want a template that is uniform from one song to the next and that ensures that the keys unique.  So, a couple of forms come immediately to mind.  One way is to simply generate a unique number for each.  For example, we could use keys of the following sort (making sure we don't use the same integer twice): 

    songs = {0:{dict for song0}, 1:{dict for song1}, ..., N:{dict for songN}}
    
Another way is to tack the integer onto to a string that denotes the entity.  For example, we could concatenate the word 'song' with an integer to create a key (again, insuring that none are duplicated):

    songs = {'song0':{dict for song0}, 'song1':{dict for song1}, ..., 'songN':{dict for songN}}
    
The code immediately below uses the first approach along with the first of the 'keying' methods. This is followed by code that utilizes the zip option along with the second 'keying' method.

In [14]:
print("Creating a Nested Dictionary using Numerical Keys and entering 'key:value' Pairs")
print("")

# using dictName[key] = value for entry and integers as keys

import pprint as pp #using this to pretty print nested dictionaries

print("List of keys for individual song dictionaries:")
print("keyList =",keyList)
print("")
print("List of lists containing the data for all songs")
print("songs = ", songs)
print("")

# create an empty dictionary to hold the individual song dictionaries
songsDict = {}

# Nested 'for loops' for creating nested dictionaries
# Outside 'for loop' cycles through each of the 'i' songs in the list
# Since each song will have it's own dictionary, we need to create these first.
# Here, each of the individual song dictionaries has a key = i
# On the inside loop we access cycle through each key and
#    retrieve the associated value in the 'songs' list of lists
# Finally, we create an entry in the individual song dictionaries for the key:value pair

for i in range(len(songs)): 
    songsDict[i] = {}
    for j in range(len(keyList)):
        tmpKey = keyList[j]
        tmpVal = songs[i][j]
        songsDict[i][tmpKey] = tmpVal
        
print('Dictionary of Songs')
pp.pprint(songsDict, depth = 2)

Creating a Nested Dictionary using Numerical Keys and entering 'key:value' Pairs

List of keys for individual song dictionaries:
keyList = ['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade']

List of lists containing the data for all songs
songs =  [[1, 'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s'], [2, 'Santana Featuring Rob Thomas', 'SMOOTH', 'Duo/Group', 'Rock', '90s'], [3, 'Bob Darin', 'MACK THE KNIFE', 'Male', 'Pop', '50s'], [4, 'Mark Ronson Featuring Bruno Mars', 'UPTOWN Funk!', 'Duo/Group', ' Pop', '10s']]

Dictionary of Songs
{0: {'Artist': 'Chubby Checker',
     'Decade': '60s',
     'Gender': 'Male',
     'Genre': 'R&B',
     'Rank': 1,
     'Title': 'THE TWIST'},
 1: {'Artist': 'Santana Featuring Rob Thomas',
     'Decade': '90s',
     'Gender': 'Duo/Group',
     'Genre': 'Rock',
     'Rank': 2,
     'Title': 'SMOOTH'},
 2: {'Artist': 'Bob Darin',
     'Decade': '50s',
     'Gender': 'Male',
     'Genre': 'Pop',
     'Rank': 3,
     'Title': 'MACK THE KNIFE'},
 3

In [15]:
print("Creating a Nested Dictionary of Songs Using String Keys and the 'zip' Function:")
print("")
# Using same keyList and songs list
# Employing zip function and concatenated keys (e.g. song1)

import pprint as pp #using this to pretty print nested dictionaries

# create an empty dictionary to hold the individual song dictionaries
songsDict = {}

# Nested 'for loops' for creating nested dictionaries
# Outside 'for loop' cycles through each of the 'i' songs in the list
# Since each song will have it's own dictionary
#   we need to create a key to designate each song dictionary
#   and to 'initialize' each of these
# On inside 'for loop' we zip the keys and values for each song
# extract each tuple from the zip object
# create each of the k:v pairs for each individual dictionary 

for i in range(len(songs)):
    tmpKey = 'song' + str(i)
    songsDict[tmpKey] = {} # initialize by creating empty dictionary
    for (k,v) in zip(keyList,songs[i]):
        songsDict[tmpKey][k] = v
     
print('Dictionary of Songs')
pp.pprint(songsDict, depth = 2)

Creating a Nested Dictionary of Songs Using String Keys and the 'zip' Function:

Dictionary of Songs
{'song0': {'Artist': 'Chubby Checker',
           'Decade': '60s',
           'Gender': 'Male',
           'Genre': 'R&B',
           'Rank': 1,
           'Title': 'THE TWIST'},
 'song1': {'Artist': 'Santana Featuring Rob Thomas',
           'Decade': '90s',
           'Gender': 'Duo/Group',
           'Genre': 'Rock',
           'Rank': 2,
           'Title': 'SMOOTH'},
 'song2': {'Artist': 'Bob Darin',
           'Decade': '50s',
           'Gender': 'Male',
           'Genre': 'Pop',
           'Rank': 3,
           'Title': 'MACK THE KNIFE'},
 'song3': {'Artist': 'Mark Ronson Featuring Bruno Mars',
           'Decade': '10s',
           'Gender': 'Duo/Group',
           'Genre': ' Pop',
           'Rank': 4,
           'Title': 'UPTOWN Funk!'}}


Notice in both cases -- individual entry or zip -- the order of the keys in the dictionary is not the same as the order in the original keyList. In the keyList it was 

    'Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade'
    
In the dictionaries it's 

    'Artist','Decade', 'Gender','Genre', 'Rank', 'Title'
    
So, what gives.

You may not recall, but earlier I said that a Python 'dictionary' is an 'unordered' collection. This basically means that it is not 'sequenced', so you cannot retrieve values based on position in large part because the position is not guaranteed from one time to the next. For instance, if we add another field to an existing dictionary, we may get a different order, even if the keys are numbers. There is a way to ensure the order by declaring that the dictionary is 'ordered'.  We'll discuss this later.

In the above examples, I've relied on the fact that the dictionaries were created from existing lists of data. As I suggested, it's a bit of a 'fudge.' With 'semi-structured' or 'unstructured' data, we rarely start with a table of values. If we did have such a table and wanted to convert it to a dictionary, we could simply read the table into a pandas' dataframe and use the 'pandas.DataFrame.to_dict' function to do the conversion. We wouldn't need a customized program.

However, with the Billboard web page, the key data was embedded in a set of 'div' tags, one for each of the 600 songs. The 'div' tag for the first song was:

    <div class="chart-list-item chart-list-item--all-time " data-rank="1" 
    data-artist="Chubby Checker" data-title="THE TWIST" data-has-content="true" 
    data-gender="Male" data-genre="R&B" data-decade="60s">

As far as tags of this sort go, it's clearly one of the easier to programmatically parse. You could use Beautiful Soup to do it, or you could do it with a set of simple string and regex functions. Either way, you could grab one of the tags to create a list of the keys or fields. Then, iterate through the tags for each of the songs, extracting the appropriate values. As you looped, you could either create the individual 'key:value' pairs one at a time or you could collect the values in a list and then zip them together with the list of keys.

When I originally analyzed the Billboard data, I did this by first creating a comma delimited file of the data that I stored on disk. Once on disk, I had the option of either reading and converting the .csv file into dataframe which in turn could be converted to a dictionary using the pandas.DataFrame.to_dict function or of using the csv.DictReader object to access the rows of the .csv file which could then be added directly to a dictionary. Although I've used both methods plus a few others, I'm only going to discuss the later.

Before I discuss how to convert a .csv file into a dictionary, we should probably take a look at how we can retrieve a list of the 'keys', 'values', or 'items' in a dictionary. There are a variety of circumstances were this comes into play. Suppose, for instance, you haven't worked with the dictionary for a while and simply want to recall it's general structure. 
Or, maybe you want to know the value of one or more fields for one or more entities (e.g. the titles of the top 10 songs).  Or, maybe you want to perform some action on a particular subset of the dictionary (e.g. adding a new field to those songs in the 'pop' genre). Or, maybe you want to add a new key, or new value to an existing key, and want to ensure that the key or value does or doesn't exist. All these cases require methods for retrieving keys, values or items.

To retrieve the 'keys', 'values', or 'items' in a dictionary, we typically use the keys(), values(), and items() methods, respectively. To retrieve the 'value' for a particular 'key:value' pair, we can use either 'dictName[key]' or the 'get' function ('dictname.get(key)').

To illustrate these methods or functions, we'll start with the directory for a sample song. Then we'll turn our attention to the nested dictionary with a sample of 4 songs.

In [16]:
# First lets start by accessing elements for an individual directory
# For this we'll use the song1D that we created earlier

print("song1D =", song1D)
print("")
print("song1D.keys():", song1D.keys())
print("")
print("song1D.values():", song1D.values())
print("")
print("song1D.items():", song1D.items())
print("")
print("song1D['Artist'] - sing the key to retrieve an associated value:", song1D['Artist'])
print("")
print("song1D.get('Artist') - using 'get(key)' to retrieve an associated value: ", song1D.get('Artist'))
print("")


song1D = {'Rank': 'Rank', 'Artist': 'Artist', 'Title': 'Title', 'Gender': 'Gender', 'Genre': 'Genre', 'Decade': 'Decade'}

song1D.keys(): dict_keys(['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade'])

song1D.values(): dict_values(['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade'])

song1D.items(): dict_items([('Rank', 'Rank'), ('Artist', 'Artist'), ('Title', 'Title'), ('Gender', 'Gender'), ('Genre', 'Genre'), ('Decade', 'Decade')])

song1D['Artist'] - sing the key to retrieve an associated value: Artist

song1D.get('Artist') - using 'get(key)' to retrieve an associated value:  Artist



While it looks like the keys(), values(), and items() methods produce a list (e.g. list of keys), they actually produce special 'dict' objects. For instance, the 'keys()' method produces a 'dict_keys' object. While these objects are 'iterable', sometimes it is helpful to convert the result to a list which can then we used for other programmatic purposes (e.g. as an index or pointer for retrieving other information).

In [17]:
songKeyList = list(song1D.keys())
print('list of songKeys:',songKeyList)
print("")
print('key in index 2 of songKeyList:', songKeyList[2])

list of songKeys: ['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade']

key in index 2 of songKeyList: Title


When dictionaries are nested, you can use the same basic methods -- keys(), values(), and items() -- but you'll need to specify multiple keys in order to gain access to inner dictionaires. We'll employ the 'songsDict' to illustrate their use in a nested case.  

In [18]:
print('keys in (outer) songs dictionary - songs.keys():')
print(songsDict.keys())
print("")
print('keys in one of the individual song dictionaries - songs[0].keys():')
print(songsDict['song0'].keys())
print("")
print('values in (outer) songs dict - songs.values():')
print(songsDict.values())
print("")
print('values in one of the individual song dict - songs[1].values()')
print(songsDict['song0'].values())
print("")
print('items in (outer) songs dict - songs.items()')
print(songsDict.items())
print("")
print('items in one of the individual song dict:', songsDict['song0'].items())
print("")
print("using 'get(key)' to retrieve a value for one of the items in songs dict - songs.get('song0') or songs['song0']")
print(songsDict.get('song0'))
print("")
print("using 'get(key)' to retrieve a value for one of the songs - songs[0'song0'].get('artist') or songs['song0']['artist']")
print(songsDict['song0'].get('artist'))


keys in (outer) songs dictionary - songs.keys():
dict_keys(['song0', 'song1', 'song2', 'song3'])

keys in one of the individual song dictionaries - songs[0].keys():
dict_keys(['Rank', 'Artist', 'Title', 'Gender', 'Genre', 'Decade'])

values in (outer) songs dict - songs.values():
dict_values([{'Rank': 1, 'Artist': 'Chubby Checker', 'Title': 'THE TWIST', 'Gender': 'Male', 'Genre': 'R&B', 'Decade': '60s'}, {'Rank': 2, 'Artist': 'Santana Featuring Rob Thomas', 'Title': 'SMOOTH', 'Gender': 'Duo/Group', 'Genre': 'Rock', 'Decade': '90s'}, {'Rank': 3, 'Artist': 'Bob Darin', 'Title': 'MACK THE KNIFE', 'Gender': 'Male', 'Genre': 'Pop', 'Decade': '50s'}, {'Rank': 4, 'Artist': 'Mark Ronson Featuring Bruno Mars', 'Title': 'UPTOWN Funk!', 'Gender': 'Duo/Group', 'Genre': ' Pop', 'Decade': '10s'}])

values in one of the individual song dict - songs[1].values()
dict_values([1, 'Chubby Checker', 'THE TWIST', 'Male', 'R&B', '60s'])

items in (outer) songs dict - songs.items()
dict_items([('song0', {'Ran

In the above examples, we've basically used these retrieval methods to document some aspect of the dictionary. Another way to use them is in 'for loops' to help guide iterative actions. To illustrate this use, we're going to take a look at the dataset for all 600 songs in the Billboard database.  As noted above, this dataset was originally stored as a .csv file. Obviously, we first need to convert it to a dictionary before we can use it to illustrate uses of the keys, values and items methods. This is what is done below:

In [19]:
import csv

csvFile = 'BB_TOP600_Songs.csv'
top600Dict = {}
cnt = 0

print("Sample Rows from DictReader:")
print("")
with open(csvFile) as crFile:
    reader = csv.DictReader(crFile)
    for row in reader:
        if cnt < 2: print(row)
        songKey = 'song' + str(cnt)
        top600Dict[songKey] = dict(row)
        cnt += 1
print("")
print("Sample Rows from Top600Dict dictionary:")
print("")
print("top600Dict['song1'] = ", top600Dict['song1'])
print("top600Dict['song2'] = ", top600Dict['song2'])

Sample Rows from DictReader:

OrderedDict([('_ID', '1'), ('Rank', '1'), ('Artist', 'Chubby Checker'), ('SongTitle', 'THE TWIST'), ('Gender', 'Male'), ('Genre', 'R&B'), ('Decade', '60s')])
OrderedDict([('_ID', '2'), ('Rank', '2'), ('Artist', 'Santana Featuring Rob Thomas'), ('SongTitle', 'SMOOTH'), ('Gender', 'Duo/Group'), ('Genre', 'Rock'), ('Decade', '90s')])

Sample Rows from Top600Dict dictionary:

top600Dict['song1'] =  {'_ID': '2', 'Rank': '2', 'Artist': 'Santana Featuring Rob Thomas', 'SongTitle': 'SMOOTH', 'Gender': 'Duo/Group', 'Genre': 'Rock', 'Decade': '90s'}
top600Dict['song2'] =  {'_ID': '3', 'Rank': '3', 'Artist': 'Bobby Darin', 'SongTitle': 'MACK THE KNIFE', 'Gender': 'Male', 'Genre': 'Pop', 'Decade': '50s'}


As noted earlier, this code employs a csv.DictReader designed to read the contents of a .csv file (in this case 'BB_TOP600_Songs.csv') one row at a time. 2 of the intermin reader rows (out of 600) are displayed above. As the display shows, the DictReader converts each row to an OrderedDict (which is the same as a regular dictionary except that it retains the order in which the key-value pairs were entered). Unless otherwise specified, the DictReader treats by default the entries in the first row of the file as the keys('Rank', 'Artist', ...) and each of the other rows as the associated values for a single entity. The keys and values are basically 'zipped' together to produce a list of 'key,value' tuples.  If you've been following along, you should recognize that applying the 'dict' function to a list of 'key,value' tuples is one way to add them to a regular (unordered) dictionary. This is what is done in the code above. For each song, the 'for loop' accesses a row in the reader, constructs a key from the word 'song' and the 'cnt', then uses the 'dict' function to create a dictionary for each song containing the 'key:value' pairs from the list of tuples in the row. The entire collection of 600 dictionaries is housed within an 'outer' dictionary labeled 'top600Dict'. Two of the rows in the top600Dict are displayed at the end.     

With the top600Dict in hand, we're ready to perform some analyses using the retrieval methods described above. 

In the first example, we're going to answer the question: how many of the top 600 songs were from the '60s' decade? Here's one way to find the answer:

In [20]:
numIn60s = 0 

for sKey in top600Dict.keys():
    if 'Decade' in top600Dict[sKey].keys() and top600Dict[sKey]['Decade'] == '60s':
        numIn60s += 1
        
print('Number of songs from the 60s:', numIn60s)

Number of songs from the 60s: 76


In this code segment, the 'for loop' cycles thru each of the 'keys' in the list of 'top600Dict' keys. For each of these 'sKey's, it first determines if (1) the 'Decade' key is in the list of keys contained in the individual dictionary for that song (designated by 'top600Dict[sKey].keys() and (2) whether the value for the 'Decade' equals the '60s'. If so, it increments the numIn60s counter. The final answer is 76 (out of the 600). The first segment of the if statement may seem superfluous because we want to make sure that song has a value for the Decade before we try to use the value. If it doesn't, and we still go ahead and error will thrown and the code will abort.

Of course, unless we've have a special interest in the 60s, more than likely we'd like to see the counts for the remaining decades as well. If we're going to use individual counters for all of the Decades, this could get a little mind numbing. First, we have to find out the number and names of the decades (there are 7 starting at the 50s and going to the 00s). Then we'd have to create 7 counters along with 7 'if' statements, 7 incrementers, and 7 print statements. Even if we decided to use an array, it would still be mind numbing.

Fortunately, Python has a 'collection' module that includes a 'Counter'.  What it enables us to do is loop thru a set of values, counting the frequency with which each individual value occurs. The results are collected into a dictionary providing the frequencies for each potential value. This is what the code might look like for this problem. 

In [21]:
import collections as cl

cnt = cl.Counter()
for sKey in top600Dict.keys():
    if 'Decade' in top600Dict[sKey].keys():
        decVal = top600Dict[sKey]['Decade']
        cnt[decVal] += 1

print("cnt:", cnt)
print("")
print("cnt keys:", cnt.keys())
print("cnt values:", cnt.values())
print("")

print('three most common Decades:', cnt.most_common(3))

cnt: Counter({'80s': 133, '70s': 114, '90s': 102, '00s': 83, '60s': 76, '10s': 71, '50s': 21})

cnt keys: dict_keys(['60s', '90s', '50s', '10s', '00s', '80s', '70s'])
cnt values: dict_values([76, 102, 21, 71, 83, 133, 114])

three most common Decades: [('80s', 133), ('70s', 114), ('90s', 102)]


After importing the 'collections' module, we establish a counter object (cnt).  This will key track of all all the individual frequences for the possible 'Decade' values. Again, the 'for loop' iterates through each of the songs which are designated by 'skey'. Next, we check to see if the 'Decade' key exists for a given song. If so, we find the value ('decVal') for it's Decade. The 'cnt' Counter keeps track of all the unique decVal values and the frequency with which they occur.

In looking at the results, it's clear that the 'Counter' is a dictionary whose 'keys' are the possible 'Decades' and whose 'values' are the frequency or counts with which the 'Decades' occur.  Given that it's a dictionary we can determine both the list of keys and the list of values.

If we were only interested in say the 3 most common 'Decades' in the dataset, we can utilize the Counters 'most_common' method to determine the answer.

<h4> Some Additional Differences between List Data Dets and Dictionary Data Sets</h4>

Thus far, except for the syntax, there's not much difference between what can be done with arrays or tables and what can be done with dictionaries.  From a programmatic standpoint, differences begin to arise as the values in a key:value pair become either more complex or more diverse in nature.

Fields or Variables with Multiple Values

As an example, let's revisit the notion of an 'artist' in the Billboard dataset. Artists can either be solo performers, duos, groups, or some combinations of each. In this source, the way in which the possibilities are designated varies, even for similar types of combinations. Here are some examples:

<ul>
    <li>Chubby Checker</li>
    <li>Santana Featuring Rob Thomas</li>
    <li>LMFAO Featuring Lauren Bennett & GoonRock</li>
    <li>Diana Ross & Lionel Richie</li>
    <li>Percy Faith And His Orchestra</li>
    <li>Bee Gees</li>
    <li>Luis Fonsi & Daddy Yankee Featuring Justin Bieber
    <li>Mariah Carey & Boyz II Men</li>
    <li>Paul McCartney And Michael Jackson</li>
    <li>Robin Thicke Featuring T.I. + Pharrell</li>
    <li>Elvis Presley With The Jordanaires</li>
    <li>Beatles With Billy Preston</li>
</ul>

So, repeating an earlier question, if you wanted to analyze particular artists in more detail, what should the data structure look like? Right now the artist is a basically a 'string' variable. I assume it's a designation provided to Billboard by the legal entity that owns the rights to the song. At the moment the only way to know whether a group or duo is involved is to look at the corresponding gender field which indicates whether it's a 'male' or 'female' artist (which implies solo) or a 'duo/group'. However, if the list contains more than one performer, then it's impossible to know just from the string which of the performers are groups and which are individuals. Additionally, the term 'Featuring' designates a minor role in the song the question. We might want to separate artists from simple contributors (although in this notebook we won't).

One easy way to represent all the artists is to use a list, splitting the artist 'string' whenever '&', 'And', 'Featuring', or 'With' appears. The code shown below does this in 3 steps (after eliminating some errant symbols): (1) replacing each of these substrings with a delimiter (in this case a pipe '|'); (2) splitting the string into a list using the delimiter; and (3) saving the result as a list in a new 'key:value' pair [i.e. top600Dict[sKey]['Performers']. For solo artists, the list consists of the original name of the artist.  In addition to creating a new 'key:value' pair for each song, a counter 'cnt' was added to determine the number of songs on which each artist performed. Only the top 25 are shown.

In [22]:
aCnt = 0
repSyms = ['(Duet ','(', ')']
splitWords = [' & ',' And ',' + ',' With ',' Featuring ']

cnt = cl.Counter()
for sKey in top600Dict.keys():
    if 'Artist' in top600Dict[sKey].keys():
        origArtist = top600Dict[sKey]['Artist']
        for rSym in repSyms:
            if rSym in origArtist:
                origArtist = origArtist.replace(rSym,'')
        sArtist = origArtist
        for sWord in splitWords:
            if sWord in sArtist:
                sArtist = sArtist.replace(sWord, '|')
        if sArtist == origArtist:
            artistList = [origArtist]
        else:
            artistList = sArtist.split('|')
        top600Dict[sKey]['Performers'] = artistList
        aCnt += 1
        for aName in artistList:
            cnt[aName] += 1
           
print('Top 25 artists with the most records:')
print(cnt.most_common(25))
print("")
totPerformersDict = dict(cnt)
top25 = dict(cnt.most_common(25))
print('Names of top25:')
print(top25.keys())
print("")
totPerformersDict = dict(cnt)
print('Total number of performers all songs combined',sum(totPerformersDict.values()))
print("")
print('Number of songs performed by top25 performers combined',sum(top25.values()))

Top 25 artists with the most records:
[('Mariah Carey', 11), ('Whitney Houston', 7), ('Katy Perry', 7), ('Madonna', 7), ('Usher', 6), ('Bee Gees', 6), ('Rihanna', 6), ('Bruno Mars', 5), ('The Beatles', 5), ('Diana Ross', 5), ('Elton John', 5), ('Alicia Keys', 5), ('Lionel Richie', 4), ('Boyz II Men', 4), ('His Orchestra', 4), ('Justin Bieber', 4), ('Paul McCartney', 4), ('Michael Jackson', 4), ('Maroon 5', 4), ('Paula Abdul', 4), ('Barbra Streisand', 4), ('Taylor Swift', 4), ('Sean Paul', 4), ('Chubby Checker', 3), ('Ed Sheeran', 3)]

Names of top25:
dict_keys(['Mariah Carey', 'Whitney Houston', 'Katy Perry', 'Madonna', 'Usher', 'Bee Gees', 'Rihanna', 'Bruno Mars', 'The Beatles', 'Diana Ross', 'Elton John', 'Alicia Keys', 'Lionel Richie', 'Boyz II Men', 'His Orchestra', 'Justin Bieber', 'Paul McCartney', 'Michael Jackson', 'Maroon 5', 'Paula Abdul', 'Barbra Streisand', 'Taylor Swift', 'Sean Paul', 'Chubby Checker', 'Ed Sheeran'])

Total number of performers all songs combined 731

Numb

Simple Text Analysis: Word Frequencies

In the initial intro to this notebook, I raised the hypothetical question of determining which words appear most frequently in the top 600 SongTitles.  While I provided the answer, I didn't indicate how I arrived at the answer. Clearly, 'counters' provide one means for calculating the most frequent words.  All we have to do is turn each title into a list of words, then apply a counter and we have our answer.

The fact that all the words are capitalized in the SongTitles simplifies the problem of having to reconcile the same words with different capitalization (e.g. AND, And, and), although the usual tactic is to convert them all to lower case. Given this, all we have to do is divide each title into a list of individual words and apply a counter. 'Regular expressions' provide one way of doing this.  The specific regex that we'll use is shown below.

I should note that this is a pretty crude analysis. The regex eliminates punction marks, replacing them with a space. This is fine for periods, commas, exclamation marks, question marks, parentheses, single quotes, and double quotes, but not so straightforward for hyphens and apostrophes. There are 5 hypens in the 600 titles. Works okay for 'HALF-BREED' and 'PEPPERMINT TWIST - PART 1' but not so well with 'UN-BROKEN' which becomes 'UN BROKEN'. Apostrophes are even more problematic, especially since they occur more frequently -- 97 in 92 songs. Apostrophes can be used for contractions (e.g. CAN'T becomes CAN T), possessives (CATHY'S becomes CATHY S), in names (L'AMOUR becomes L AMOUR), and slang (FALLIN' for FALLING). In real text analysis each of these cases requires special treatment. There are ways to craft one or more regular expressions to handle these cases. However, it's much too detailed to cover here.

In [23]:
import re

cnt = cl.Counter()
for songKey in top600Dict.keys():
    titleTxt = top600Dict[songKey]['SongTitle']
    titleWords = re.findall(r'\w+', titleTxt)
    for word in titleWords:
        if len(word) > 1:
            cnt[word] += 1
        
print('Most Frequent Words in Song Titles (Excluding 1 letter words):')
print("")
print(cnt.most_common(10))

Most Frequent Words in Song Titles (Excluding 1 letter words):

[('THE', 76), ('YOU', 75), ('LOVE', 50), ('ME', 41), ('TO', 37), ('IT', 31), ('OF', 26), ('MY', 26), ('IN', 25), ('THAT', 16)]


Similar kinds of analysis can be applied to larger bodies of text, although multiple counters are required, different word forms are also analyzed, and often interim text analysis results are stored with each entity to speed up further analysis. For instance, the Billboard data included a description with the top 100 songs but not the remaining 500. Like the other data, the descriptions were also contained in separate a 'div' tag in the underlying HTML code. As an example, here's the entry for THE TWIST:
<br>
>Checker's transformative smash remains the No. 1 all-time Billboard Hot 100 hit, fueled by its dual run atop the weekly chart in September 1960 (for one week) and January 1962 (for two frames). "The Twist" is the only song to rule the Hot 100 in separate release cycles, as it returned to the top after adults caught on to the song and its namesake dance that younger audiences had first popularized. "That style of dancing wasn't there before," Checker, now 76, mused. "It was so explosive, it's never left the dance floor."</div>

Again, if we were interested in doing some sort of text analysis on the descriptions (e.g. do they differ by decade?), we could add it to those dictionaries of the top 100 song, ignoring it for the others. In the case of THE TWIST, the added entry might look like this:

    song[0]['description'] = 'Checker's transformative smash ... never left the dance floor.'

Once the description was added, we could perform various sorts of text analysis on each of the descriptions, storing interim and final results with each song. For example, we might introduce a 'text analysis' key for each of the songs that has a description and use a nested dictionary to maintain various lists of results, e.g. 

    song[0]['text analysis'] = {'tokens':[list of tokens], 'nonStopWords':[list of nonStops],'lemmas':[list of lemmas],
                                'bigrams':[lists of bigrams],...}.

<h4>Other Methods</h4>

Like Python lists, there are a wide variety of methods that can be used to modify, explore and analyze dictionaries in various ways. This discussion has only consider the representation and creation of dictionaries along with some simple forms of analysis. For those who are interested in more details about the other methods, a good place to start is<a href="https://python-reference.readthedocs.io/en/latest/docs/dict/"> Python Reference: The Right Way</a>.

<a class="anchor" id="JSON"></a>
<h3>JSON</h3>
<a href='#toc'>Return to TOC</a>

Like most other objects in Python, dictionaries only exist in memory.  Once the associated computer is turned off, the dictionaries disappear along with all the other objects. With dictionaries there are ways to create more persistent versions. Ignoring databases for the moment, in Python two of the standard ways to do this are: 

<ul>
    <li><i>Pickling</i> -- Pickling is a form of serialization. In pickling a Python object is converted to a byte stream. Lots of Python data types and objects can be pickled including: None, True, False, integers, floating point numbers, complex numbers, strings, bytes, bytearrays, tuples, lists, sets, and dictionaries containing only picklable objects. The reverse of pickling is <i>unpickling</i> which is a form of deserialization that converts a byte stream back into an object hierarchy. To write a pickled object to an "open" file one uses the 'pickle.dump(python_object,pickle_file) function. To read an existing binary pickle file and convert the file back to an object (hierarchy) one uses the pickle.load(pickle_file) function.</li><br>
    <li><i>JSON Serialization</i> -- While JSON is an abbreviation for JavaScript Object Notation, it's really a language independent, text-based, data interchange format. In JSON, serialization objects are encoded into strings that can be written to and read from files or interchanged across a network.  Virtually any program can read, write or interchange JSON strings. In Python a number of objects can be be serialized via the JSON module into JSON format and deserialized into a Python object hierarchy. The json.dump(obj,file) function is used to write an Python object to a .json file, while json.load(file) is used to convert the .json file to a Python object.</li>
</ul>

The following table summarizes the differences between the two forms of serialization:
<br>
<table>
<tr><th>Property/Feature</th><th>Pickle</th><th>JSON</th></tr>
<tr><td>serialization</td><td>Binary</td><td>Text</td></tr>
<tr><td>human-readable</td><td>No</td><td>Yes</td></tr>
<tr><td>interoperable</td><td>Python Specific</td><td>Language Agnostic</td></tr>
<tr><td>object support</td><td>Extensive</td><td>Subset</td></tr>
<tr><td>writing to file</td><td>pickle.dump</td><td>json.dump</td></tr>
<tr><td>reading from file</td><td>pickle.load</td><td>json.load</td></tr>
</table>

Note: I've used "pickle" with a number of projects.  Usually, it's served as a backup holding tank for interim results, as well as a way to share results from one Python program or project to the next.  As the table indicates, it doesn't provide a way to share results among programs written in different languages (e.g. Python and Javascript), nor can it act as a feed to various databases (like MongoDB).

The following code illustrates how simple it is to use the JSON module to write the songsDict dictionary to a .json file and subsequently to read it back into a dictionary. Recall, the songsDict only has 4 songs in it (to reduce the size of the i/o for this example). In this case the 'json.dumps' method is first used to write the dictionary to a string variable 'jsonStr'. This variable is then written in raw format to the songsDict.json file.  It's also possible to serialize the dictionary and to add indentation to improve it's written appearance, as well as sorting the keys to put the output in alphabetic order. Both these make it easier for a human to read.

In [24]:
import json

# raw output
jsonFile = 'songsDict.json'
fOpen = open(jsonFile, 'w')
jsonStr = json.dumps(songsDict)
fOpen.write(jsonStr)
fOpen.close()

print('type(jsonStr) =', type(jsonStr))
print("")
print("jsonStr:")
print(jsonStr)

# indented output with sorted keys

jsonFile = 'songsDict.json'
fOpen = open(jsonFile, 'w')
jsonStr = json.dumps(songsDict, sort_keys=True, indent = 4)
fOpen.write(jsonStr)
fOpen.close()
print('Sorted Order and Indented Output')
print(jsonStr)

536

type(jsonStr) = <class 'str'>

jsonStr:
{"song0": {"Rank": 1, "Artist": "Chubby Checker", "Title": "THE TWIST", "Gender": "Male", "Genre": "R&B", "Decade": "60s"}, "song1": {"Rank": 2, "Artist": "Santana Featuring Rob Thomas", "Title": "SMOOTH", "Gender": "Duo/Group", "Genre": "Rock", "Decade": "90s"}, "song2": {"Rank": 3, "Artist": "Bob Darin", "Title": "MACK THE KNIFE", "Gender": "Male", "Genre": "Pop", "Decade": "50s"}, "song3": {"Rank": 4, "Artist": "Mark Ronson Featuring Bruno Mars", "Title": "UPTOWN Funk!", "Gender": "Duo/Group", "Genre": " Pop", "Decade": "10s"}}


770

Sorted Order and Indented Output
{
    "song0": {
        "Artist": "Chubby Checker",
        "Decade": "60s",
        "Gender": "Male",
        "Genre": "R&B",
        "Rank": 1,
        "Title": "THE TWIST"
    },
    "song1": {
        "Artist": "Santana Featuring Rob Thomas",
        "Decade": "90s",
        "Gender": "Duo/Group",
        "Genre": "Rock",
        "Rank": 2,
        "Title": "SMOOTH"
    },
    "song2": {
        "Artist": "Bob Darin",
        "Decade": "50s",
        "Gender": "Male",
        "Genre": "Pop",
        "Rank": 3,
        "Title": "MACK THE KNIFE"
    },
    "song3": {
        "Artist": "Mark Ronson Featuring Bruno Mars",
        "Decade": "10s",
        "Gender": "Duo/Group",
        "Genre": " Pop",
        "Rank": 4,
        "Title": "UPTOWN Funk!"
    }
}


<h3>MongoDB: Persisting and Analyzing Python Dictionaries and JSON Files</h3>

In the world of data analysis, if you're working with a small collection of tables for a limited period of time, and you want to persist the data and potentially use it with other programs (in the same or a different language), then .csv files and spreadsheets will suffice. When you have moderate collection of tables that will be used for longer periods of time (potentially by other people) and you need to manage them in an orderly and secure fashion, then it makes sense to store the tables in a relational (SQL) database that provides management, manipulation, and analytical capabilities.

It's the same way with semi-structured, hierarchical data. If you're working with a small collection of dictionaries, hash tables, associative arrays other objects, and you want to persist them so you can use them later in the same or other programs, then JSON files will suffice. However, as the collection grows, use continues, and the complexity increases, the need for a database management system that provides for the management, manipulation, and analysis of this semi-structure data arises. Whiles it's possible to 'shoehorn' this data into a relational schema, it often makes much more sense to use a system that handles the data 'natively.' One option is to used a 'document' database. 

While there are a number of document databases available, the 'NoSQL' database known as <i>MongoDB</i> seems to have the biggest market penetration.

>"MongoDB is a document store, and the current top NoSQL database engine in use today. As is the requirement for NoSQL engines, MongoDB does not use a relational schema, instead using JSON-like "documents" to store data. The document is akin to a record, housing fields and values. MongoDB supports dynamic schemas, and is free and open source software."

The term <i>document</i> does not refer to a text document (like a paper, or PDF, or wordfile, or book, etc.). Instead, as this quote notes, it refers to a JSON-like object used to represent, format and store semi-structured and unstructured objects and data. More technically, in MongDB the JSON-like object is stored as a BSON object. "BSON ('Binary JSON') is a binary serialization of JSON-like documents ... that also contains extensions that allow representation of data types that are not part of JSON."

In the context of this notebook and discussion, MongoDB provides a natural extension to Python and JSON for managing and manipulating dictionaries because:

<ol>
    <li>MongoDB can import JSON files</li>
    <li>Python can connect directly to MongoDB thru the PyMongo module so that we can invoke  CRUD capabilities directly from a program.</li>
</ol>

<h4>Comparing MongoDB and Relational Terminology</h4>

The vast majority of data analysts and scientists are familiar with the terminology used to describe the key conceptual objects in a relational (SQL) database. A quick summary of the basic relational terms (i.e. database, table, row, column, etc.) is provided below.  Next to each term, in the third column of the table, is the basic set of terms used with MongoDB.  On a conceptual level, they match the relational terms. Like relational databases the top level grouping is the 'database.' The same is true for MongoDB. In MongoDB each database consists of a group of related 'collections', while a relational database consists of 'tables' and views (which are also tables). Within the collections of MongoDB are various 'documents' which serve the same basic purpose as a row on the relational side. In a relational table, the rows often contain data elements (i.e. field or attribute values) for a given entity or construct (e.g in the Billboard example each row contains field attributes for a single song). The same is true for document ini Mongo which contains the attribute or field values for a given entity. In a relational database, the table columns represent the attributes, properties or fields of interest. In Mongo the keys of a document represent the same thing. Finally, in relational database the user (administrator) specifies one or more of the columns to be used in generating a unique primary key for each entity or row in the table. In MongoDB the individual documents are uniquely distinguished by a primary key that is either set to an '\_id' field in the document or automatically generated by the system.

<table >
    <tr><td><b>SQL DB Terms</b></td><td><b>MongoDB Terms</b></td></tr>
	<tr><td>database</td><td>database</td></tr>
	<tr><td>table</td><td>collection</td></tr>
	<tr><td>row</td><td>document or BSON document</td></tr>
	<tr><td>column</td></td><td>field</td></tr>
	<tr><td>index</td></td><td>index</td></tr>
	<tr><td>table joins</td><td>embedded documents and linking</td></tr>
	<tr><td>primary key<br>Unique column or<br>column combination as primary key</td></tr>
    <td>primary key<br>In MongoDB<br>the primary key is automatically <br>set to the _id field</td></tr>
</table>

In spite of the similarities in their conceptual terminology, there is still a couple of overriding differences between the two types of databases. NoSQL databases like MongoDB are essentially 'schemaless', while relational databases are 'schema bound' (if there is such a term). When you create a relational table, the type, size and order of the columns is defined and fixed ahead of time along with the primary key. While the order in which you enter rows is usually unimportant,the order and type of the values that are entered is explicitly fixed by the schema definition of the columns. Any changes, such modifying the type or name of a column, are also done with explicit commands. In MongoDB, you don't specify the document and field schema ahead of time. You simply name your database and the collection you are going to create, then you entering the documents in the collection as you see fit. Once the documents are entered they are available to be implicitly modified, queried and analyzed without an explicit command. 

Of course, the fact that the collections and documents are 'schemaless' doesn't mean that you shouldn't layout a general schema ahead of time. In fact in most MongoDB applications the schema is pretty well thought out ahead of time. Yet, even with a well-defined schema, the structure of each document can be pretty flexible, and it doesn't have to be fixed from one entity to the next. In the same vein, if there are keys or attributes that need to be added or the types of the keys need to be changed, it doesn't take an act of congress to do it. For example, if you want to change the value of a 'key:value' pair from a string to a list or a list to a dictionary or tuple, you simply enter the new value. For example,

    dictName[key] = [values] to dictName[key] = {values}

So, let's dive into MongoDB. We're going to start by building a MongoDB database with a single collection. In this way we can add substance to the terminology. Again, I'm only going to scratch the surface of MongoDB, discussing and illustrating how MongoDB, Python dictionaries and JSON fit together for purposes of analyzing semi-structured data. If your interests are broader, there are several books that cover and illustrate the myriad of details of the system. The <a href = "https://docs.mongodb.com/manual/"><i>MongoDB documentation/manual</i></a> also provides a very good introduction to these details. 

<h4>MongoDB: Creating a Database and Collection</h4>

MongoDB comes in a variety of versions that run as a service on the Cloud, on larger servers, or on PCs/MACs.  While MongoDB.com encourages folks on Windows to shift to the Enterprise or Azure editions, the "community' edition works well for an 'armchair' data analyst or scientist like myself. The 'community' version that I currently have on my PC is 4.0.

I'm going to assume that MongoDB is installed and up and running. Shortly, I'll show you how to access MongoDB directly from Python. Before I do this, however, I'm going to give you a cursory overview of  a few of the key CRUD (create, read, update, and delete) operations using the <i>Mongo shell</i> to them. The Mongo shell is an interactive JavaScript interface to MongoDB that is provided with the system. The shell supports both end user and administrative operations, although many folks employ other (more user friendly) applications to carry out these tasks. At the moment, MongoDB can't be run directly in a Jupyter notebook, so I'm going to 'mock up' the shell i/o and any other Mongo commands I need.

To activate the shell we simply run 'mongo.exe' in a command window or one of its equivalents (depending on the operating system).  The shell will respond with the mongoDB shell and server versions as well as information about the server host (127.0.0.1) and port (27017) to which the shell is connected.

    >mongo.exe
    
    MongoDB shell version v4.0.0
    connecting to: mongodb://127.0.0.1:27017
    MongoDB server version: 4.0
    ...
    >

Before we 'create' a database, one of the first things we might do is determine the databases that already exist. This is accomplished with the 'show dbs' command:

    > show dbs
    admin   0.000GB
    census  0.000GB
    config  0.000GB
    local   0.000GB
    top600  0.000GB
    
There are 5 databases including 3 adminstrative databases (admin, config and local) and 2 user defined (census and top600). The top600 contains the data we've been working with all along. To see what (document) collections exist in this particular database, or any other MongoDB database, we use the 'show collections' command. Before we do this, however, we need to focus on the particular database whose collections were interested in.  This is done with the 'use command'. The combination looks like this. 

    > use top600
    switched to db top600
    > show collections
    songs

The top600 database consists of a single collection of documents called songs. 

Note: If you're familar with the MySQL relational database and its client or shell, then this sequence of commands should look familar. There the equivalent commands are SHOW DATABASES, USE <dbName>, and SHOW TABLES.

How many documents are in the songs collection? We can answer the question by issuing a count command. The answer confirms what we already know - there are 600 songs in the database. 

    > db.songs.count()
    600

If we want to see the contents of a portion of the songs documents, we can use the 'find' command.

    > db.songs.find()

    { "_id" : 1, "Rank" : 1, "Artist" : "Chubby Checker", "SongTitle" : "THE TWIST", "Gender" : "Male", "Genre" : "R&B", "Decade" : "60s" }
    { "_id" : 2, "Rank" : 2, "Artist" : "Santana Featuring Rob Thomas", "SongTitle" : "SMOOTH", "Gender" : "Duo/Group", "Genre" : "Rock", "Decade" : "90s" }
    ...
    Type "it" for more
    >

If you remember the top600 Python dictionary from earlier in the notebook, you may notice that there is a new 'key:value' pair in each of these songs documents, e.g. "\_id": 1 and "\_id": 2. In MongoDB when a new document is inserted in a collection the system automatically creates an index on the document's \_id <i>primary key field</i>. If the \_id key already exists in a document, then MongoDB will use it, assuming it is unique.  If it doesn't exist, it will create one and assign its own value (which usually looks something like this example "\_id" : ObjectId("577f9cecd71d71fa1fb6f43a")).
Not something you want to type very often but clearly it's unique.

The next logical questions that ought to come to mind should be: (1) how was the 'top600' database created in the first place? (2) how did we create the 'songs' collection? and (2)how did we 'insert' the 600 songs in the collection?

In a relational database, the answer would be that there's a CREATE DATABASE, CREATE TABLE, and an INSERT command. In MongoDB there are no explicit CREATE commands for databases or collections, however there is a database command that operates like much like the INSERT command. As a trivial example, consider the following commands:

    > show dbs
    admin   0.000GB
    census  0.000GB
    config  0.000GB
    local   0.000GB
    top600  0.000GB
    > use testdb
    switched to db testdb
    > db.testcollect.insert({'tname':'dave','tage':'65+'})
    WriteResult({ "nInserted" : 1 })
    > show dbs
    admin   0.000GB
    census  0.000GB
    config  0.000GB
    local   0.000GB
    testdb  0.000GB
    top600  0.000GB
    >> db.testcollect.find()
    {"_id" : ObjectId("5c10144be9c65f9733dc639f"), "tname" : "dave", "tage" : "65+" }
    >

In MongoDB when we issue a 'use db' command and the database doesn't exist, MongoDB waits for the first 'insert' commmand. When this occurs, it automatically creates the database (here 'testdb'), the collection (i.e. 'testcollect'), and the first document along with its primary key (i.e. \"\_id" : ObjectId("5c10144be9c65f9733dc639f")). In a nutshell, database and collection creation are done implicitly.

With the 'top600' database we could have done the same sort of things, entering the documents one at a time with a db.collection.insert({individual document) command, or we could have created an MongoDB array and assigned it to a variable which was then used with a db.collection.insertMany(varName) command.  But, I 'fudged' again and simply used a special program called 'mongoimport' that runs outside the MongoDB shell.  In this particular case, the command looked like this:

    mongoimport -d top600 -c songs --file BB_Top600_Songs.csv --type csv --headerline

In this instance, the program imported a '.csv' file called 'BB_Top600_Songs.csv'. Because the '--headline' setting was included, the first row was used to establish a common set of keys for each of the documents and to indicate whether the primary '\_id' index was also provided with each document. The values in the remaining rows were were paired (i.e. zipped) with the appropriate keys to create each of the 600 documents. The documents were stored in a ('-c') collection called 'songs' which were housed in the designated ('-d) database 'top600'. Neither the database nor the collection existed prior to the import.  As an alternative, we could have also imported the 'songs.json' file. The main differences would be the file name extension (.json) and the type (json versus cvs), although it would have also required modifications in the way we handled the primary index.  

<h4>Querying the Collections in a Database</h4>

MongoDB supports a wide variety of commands. Many of these take the form illustrated above:

    db.<collectionName>.<commandName>(<parameters>)
    
This includes the insert, count, find, update, modify, delete, and dropDatabase commands. While all of these are important, the <i>find</i> and <i>count</i> commands are a couple of commands that are used frequently in data analysis. Even though it's syntax varies, the 'find" command is similar to the SELECT command in the relational world. Like the SELECT command, it's used to query a collection in order to extract documents that fulfill specified conditions. In MongoDB, the specified conditions are known as "query filter parameters". The major difference between 'find' and SELECT is that SELECT supports 'GROUP BY' operations while 'find' does not. In MongoDB "GROUP BY' operations are handled with 'Aggregation' operations that are separate from 'find'.

Sample Queries

The general structure of the find command is:

    db.<collectionName>.find({<query filters>}
    
This is equivalent to 

    SELECT <field_list> FROM <tableName> WHERE <query filters>
      
In MongoDB the <i>query filters</i> are represented as JSON documents which can run the gamut from simple to complex, depending on your interests.

We've already seen the simplest of all the 'find' queries. The one that selects all the documents. Technically, this statement should look like 'db.songs.find({})' with a blank JSON document as the query filter, but it also works with no document specification. The following table provides examples of some common 'comparison' and 'logical' filter conditions:

<table>
    <tr><th>Name</th><th>Matches Values</th><th>Example</th></tr>
    <tr><th>Comparison</th><th></th><th></th></tr>
    <tr><td>\$eq</td><td>equal to a specified value</td><td>db.songs.find({Decade:"60s"})</td></tr>
    <tr><td>\$ne</td><td>not equal to a specified value</td><td>db.songs.find({Decade:{\$ne:"60s"}})</td></tr>      
    <tr><td>\$gt</td><td>greater than a specified value</td><td>db.songs.find({Rank:{\$gt:590}})</td></tr>
    <tr><td>\$gte</td><td>greater than or equal to a specified value</td><td>db.songs.find({Rank:{\$gte:590}})</td></tr>
    <tr><td>\$lt</td><td>less than a specified value</td><td>db.songs.find({Rank:{\$lt:590}})</td></tr> 
    <tr><td>\$lte</td><td>less than or equal to a specified value</td><td>db.songs.find({Rank:{\$lte:590}})</td></tr>
    <tr><td>\$in</td><td>any of the values specified in an array</td><td>db.songs.find({Decade:{\$in:["60s","70s"]}})</td></tr> 
    <tr><td>\$nin</td><td>none of the values specified in an array</td><td>db.songs.find({Decade:{\$nin:["60s","70s"]}})<td></td></tr>
    <tr><td>/substring/</td><td>substring is part of a string value</td><td>db.songs.find({Artist:{/Paul McCartney/})</td></tr>
    <tr><td>\$regex</td><td>satisfies 'regular expression' condition</td><td>db.songs.find({Genre:{\$regex:/^R/}})</td></tr>
    <tr><td>\$text</td><td>if one or more fields has a 'text index', contains any terms in search string, and satisfies search flags</td><td>db.songs.find({\$text:{\$search: "LOVE LOVES", \$caseSensitive: true}})</td></tr>
    <tr><th>Logical</th><th></th><th></th></tr>
    <tr><td>/substring/i</td><td>ignoring case, substring is part of a string value</td><td>db.songs.find({Artist:{/Paul McCartney/i})</td></tr>
    <tr><td>cond1,cond2...</td><td>when all conditions are satisfied-similar to '\$and'</td><td>db.songs.find(Artist:{/Paul McCartney/i},Decade:"70s"})</td></tr>
    <tr><td>\$and</td><td>when all conditions are satisfied</td><td>db.songs.find({\$and:[{Artist:/Paul McCartney/i,Decade:"70s"}]})</td></tr>
    <tr><td>\$or</td><td>when any condition is satisfied</td><td>db.songs.find({\$or:[{Artist:/Paul McCartney/i,Artist:"The Beatles"}]})</td></tr>
    <tr><td>\$not</td><td>when condition is not satisfied</td><td>db.songs.find({Rank:{$not:{$gt:10}}})</td></tr>
    <tr><td>\$nor</td><td>when all conditions are false</td><td>db.songs.find({$nor:[{Rank:{$lt:590}},{Decade:"60s"}}]})</td></tr>
</table>

Note: this table is a relatively small subset of the entire list of query operators supported by MongoDB. A complete list can be found in their <a href="https://docs.mongodb.com/manual/reference/operator/query/">reference documents</a> on the subject. 

<h4>Querying Nested/Embedded Documents and Fields</h4>

Thus far, none of the examples that are provided contain 'nested fields'. By this we obviously mean a field that contains other fields. The reason for this oversight is that none of the collections in the 'top600' database contain nested fields. We did touch on this subject a bit earlier in the discussion about creating a 'Performers' field out of the 'Artist' field because the 'Artist' field is really a string that contains solo performers, duos, groups, and various combinations of these. For example, the 'Artist' for the 6th ranked song was "LMFAO Featuring Lauren Bennett & GoonRock" which includes the duo 'LMFAO' and featured two solo performers Lauren Bennett and GoonRock. Or, the 'Artist' for the 18th ranked song "Diana Ross & Lionel Richie" which includes two solo performers. So, given these sorts of possibilites we might add a field to the songs collection called "Performers" which in turn has two fields "Lead" and "Featured".

    {"Performers":{"Lead":[listofMainArtists],"Featured":[listofFeatured]}}
    
With this nested field, we could represent the two 'Artist's for the 6th and 18th ranked songs as:
     
    {"Performers":{"Lead":["LFMAO"],"Featured":["Lauren Bennett","GoonRock']}}
    {"Performers":{"Lead":["Diana Ross","Lionel Richie"]}
    
To illustrate a few of the features of nested fields, I went ahead and created a second collection called 'songArtists' with 10 documents. Here's what they look like:
    
    # Display all the SongArtists documents
    >db.songArtists.find()
    {"_id":1,"Performers":{"Lead":["ChubbyChecker"]},"SongTitle":"THETWIST","Decade":"60s"}
    {"_id":2,"Performers":{"Lead":["Santana"],"Featured":["RobThomas"]},"SongTitle":"SMOOTH","Decade":"90s"}
    ...
    {"_id":10,"Performers":{"Lead":["PaulMcCartney","StevieWonder"]},"SongTitle":"EBONYANDIVORY","Decade":"80s"}

For the moment, we're only interested the "Performers" field, so let's list the documents, but only show the id and Performers fields. This is done by providing a list of fields to be included or those to be excluded (note: by default the \_id is always included unless specified otherwise).

    # Display _id by default and show Performers field
    > db.songArtists.find({},{Performers:1})
    { "_id" : 1, "Performers" : { "Lead" : [ "Chubby Checker" ] } }
    { "_id" : 2, "Performers" : { "Lead" : [ "Santana" ], "Featured" : [ "Rob Thomas" ] } }
    ...
    { "_id" : 10, "Performers" : { "Lead" : [ "Paul McCartney", "Stevie Wonder" ] } }
 
    # Show the Performers field and exclude _id
    > db.songArtists.find({}, {Performers:1, _id:0})
    { "Performers" : { "Lead" : [ "Chubby Checker" ] } }
    { "Performers" : { "Lead" : [ "Santana" ], "Featured" : [ "Rob Thomas" ] } }
    ...
    { "Performers" : { "Lead" : [ "Paul McCartney", "Stevie Wonder" ] } }
 
In order to see a field within an embedded or nested document, you reference it with a '.' dot between the two fields. For example, to see all the 'Lead' artists in the 'Performers' document, the designation would be 'Performers.Lead'.  When you do this, the combination must be enclosed in quotes ("...") within the query. 
 
     # Display the _id by default and the Lead (artists) in the Performers field
     > db.songArtists.find({},{"Performers.Lead":1})
    { "_id" : 1, "Performers" : { "Lead" : [ "Chubby Checker" ] } }
    { "_id" : 2, "Performers" : { "Lead" : [ "Santana" ] } }
    ...
    { "_id" : 10, "Performers" : { "Lead" : [ "Paul McCartney", "Stevie Wonder" ] } }
    
In the 'songArtists' collection every nested 'Performers' document has 'Lead' artists. In contrast 'Featured' artists only appear in subset of these documents (i.e. not all songs have 'Featured' artists). In setting up this dataset we could have chosen to include the 'Featured' field and assigned those with no 'Featured' artists a value like 'null' or 'NA'. However, I chose to simply omit it instead (which is one of the benefits on using documents rather than tables). As a consequence, when you issue a blanket query to 'find' the 'Performers' who are featured, the listing will include every 'Performers' document whether it has a 'Featured' field or not.

    # Display the Featured (artists) in the Performers field and exclude _id
    > db.songArtists.find({},{"Performers.Featured":1, _id:0})
    { "Performers" : {  } }
    { "Performers" : { "Featured" : [ "Rob Thomas" ] } }
    { "Performers" : {  } }
    { "Performers" : { "Featured" : [ "Lauren Bennett", "GoonRock" ] } }
    { "Performers" : {  } }
    { "Performers" : { "Featured" : [ "Justin Bieber" ] } }
    ...
    { "Performers" : {  } }

To avoid this, you can use the $exists operator to determine whether the field exists and only list the document if this is true.

    # If they exist, Display the Featured (artists) in the Performers field and exclude _id
    > db.songArtists.find({"Performers.Featured":{$exists:true}},{"Performers.Featured":1, _id:0})
    { "Performers" : { "Featured" : [ "Rob Thomas" ] } }
    { "Performers" : { "Featured" : [ "Lauren Bennett", "GoonRock" ] } }
    { "Performers" : { "Featured" : [ "Justin Bieber" ] } }

<h4>Aggregation Operations</h4>

The general format of a relational 'SELECT' command is:

    SELECT <fieldList> FROM <table(s) or view(s)> WHERE <selectionCriteria> 
        GROUP BY <groupbyFieldList> HAVING <groupbySelectionCriteria> 
        ORDER BY <orderbyFieldList> (ASC|DESC)
        
As we noted above, in MongoDB the 'find' command provides equivalent features and functions as 'SELECT' with the exception of the aggregation functionality provided by GROUP BY and HAVING. Instead, MongoDB provides this functionality in a separate set of 'aggregation' operations.  These operations "group values from multiple from multiple documents together, and can perform a variety of operations on the grouped data to return a single result." Actually, MongoDB provides three sets of aggregation operations including the aggregation pipeline, the map-reduce function, and single purpose aggregation methods. 

For our purposes, we going to limit this discussion to the 'aggregation pipeline'. Even here, we're only going to touch the surface.  All of the sets of aggregation operations are very extensive. Again, for details see the <a href="https://docs.mongodb.com/manual/aggregation/">MongoDB Documentation on Aggregation.</a>

<h5>Aggregation Pipeline</h5>

MongoDB’s aggregation pipeline is a data framework analogous to the conept of a 'dataprocessing pipeline'. Basically, documents pass through a pipeline of one or more linked stages. Each stage transforms the documents and passes them to the next stage. The final outcome is an aggregated result. Formally, the stages are defined with an array whose syntax is: 

    db.collection.aggregate( [ { <stage> }, ... ] )

The list of possible stages includes:

    $addFields, $bucket, $bucketAuto, $collStats, $count, $facet, $geoNear, $graphLookup, $group,
    $indexStats, $limit, $listSessions, $lookup, $match, $out, $project, $redact, $replaceRoot,
    $sample, $skip, $sort, $sortByCount, $unwind
    
Like most things in MongoDB, each stage is a document whose syntax is:

    {$stageName:{parameters}}
    
The most basic pipeline stages provide filters that operate like queries and document transformations that modify the form of the output document. Other pipeline operations provide tools for grouping and sorting documents by specific field or fields as well as tools for aggregating the contents of arrays, including arrays of documents. In addition, pipeline stages can use operators for tasks such as calculating the average or concatenating a string. The MongoDB documentation provides a <a href="https://docs.mongodb.com/manual/meta/aggregation-quick-reference/#stages">Quick Reference Guide</a> that lists and defines all the stages with examples.

<h5>Examples of the Aggregation Pipeline</h5>

The following examples illustrate the use of the stages for 'group by' purposes:

<ol>
<li>For all songs, display the number (count) of songs (documents) by Decade: This is a single stage pipeline that is similar to the "Counter" object in Python. Here, the specification:
        
        _id:$"fieldName" 
        
delineates the 'group by' field. In this case, the computation being performed is a simple count of the total number of documents in the group.

    > db.songs.aggregate({$group:{_id:"$Decade", count:{$sum:1}}})
    { "_id" : "70s", "count" : 114 }
    { "_id" : "80s", "count" : 133 }
    { "_id" : "10s", "count" : 71 }
    { "_id" : "00s", "count" : 83 }
    { "_id" : "90s", "count" : 102 }
    { "_id" : "50s", "count" : 21 }
    { "_id" : "60s", "count" : 76 }

</li><br>

<li>For those songs whose SongTitles contain the string 'LOVE' display the number (count) of songs by Decade: This is a multi-stage pipeline containing the \$match, \$project, and \$group stages. The $match stage is like a 'find' query. Here, the query operator is a regular expression (\$regex). In regular expressions '.*' pattern matches any set of characters in a string regardless of what they are. This expression uses one to match the characters before the string 'LOVE' and another for the characters after. The problem is that '.' is a reserved character in MongoDB. Sok, in order to actually use the pattern we have to specify an 's option' telling MongoDB to ignore it. The result from the \$match is a subset of songs with the word 'LOVE' in the title. This subset still has all the fields from the original collection. The next stage is \$project which specifies a subset of fields to be used in the following stage. In this case it's the _id, SongTitle, and Decade. The last stage is \$group. As noted this stage delineates the 'group by' field(s) and the computation to be performed.  Again, it's a simple count. 

    > db.songs.aggregate(
    {$match:{SongTitle:{$regex: /.*LOVE.*/,$options:"s"}}},
    {$project:{SongTitle:1, Decade:1,  _id:1}},
    {$group:{_id:"$Decade", count:{$sum:1}}})
    { "_id" : "60s", "count" : 9 }
    { "_id" : "70s", "count" : 12 }
    { "_id" : "50s", "count" : 1 }
    { "_id" : "00s", "count" : 6 }
    { "_id" : "90s", "count" : 14 }
    { "_id" : "80s", "count" : 12 }
    { "_id" : "10s", "count" : 3 }
    
</li><br>

<li>For all songs calculate average Rank and count by Genre and display descending order by average Rank: Again, this is a multi-stage aggregation involving the \$project, \$match and \$sort stages. The \$project stage ensures that the Rank and Genre fields are in the subset of fields passed to the \$group stage. The \$Group stage specifies that we want to calculate the \$avg Rank and the count for each Genre. Finally, the \$sort stage sorts the avgRanks by descending (-1) order. 

    > db.songs.aggregate(
        {$project:{Rank:1, Genre:1, _id:1}},
        {$group:{_id:"$Genre", avgRank:{$avg:"$Rank"},count:{$sum:1}}},
        {$sort:{avgRank:-1}})
    { "_id" : "Jazz", "avgRank" : 366.2, "count" : 5 }
    { "_id" : "Pop", "avgRank" : 314.51666666666665, "count" : 240 }
    { "_id" : "R&B", "avgRank" : 304.28387096774196, "count" : 155 }
    { "_id" : "Rock", "avgRank" : 297.1666666666667, "count" : 108 }
    { "_id" : "Country", "avgRank" : 285.95652173913044, "count" : 23 }
    { "_id" : "Hip-Hop/Rap", "avgRank" : 270.71794871794873, "count" : 39 }
    { "_id" : "Dance/Electronic", "avgRank" : 228.85185185185185, "count" : 27 }
    { "_id" : "Latin", "avgRank" : 137.66666666666666, "count" : 3 }

</li><br>

Note: There is a very extensive list of <a href="https://docs.mongodb.com/manual/reference/operator/aggregation/  ">aggregation pipeline arithmetic operators</a> that can be employed in the \$group stage as well as other stages available in the MongoDB documentation.

<li>Save the avgRank results generated in the last example to a new 'collection': This is the same as the last pipeline except that an \$out stage has been added.  In the \$out stage the results are saved to a new 'collection' called 'avgRanks'. To demonstrate that the $out stage actually worked I've displayed the list collections after the addition and queried the new 'avgRanks' collection to display its contents.

    > db.songs.aggregate( 
        {$project: {Rank:1, Genre:1, _id:1}},
        {$group:{_id:"$Genre", avgRank:{$avg:"$Rank"},count:{$sum:1}}},
        {$sort:{avgRank:-1}},
        {$out:"avgRanks"})
        
    > show collections
    avgRanks
    reviews
    songArtists
    songs
    
    > db.avgRanks.find()
    { "_id" : "Jazz", "avgRank" : 366.2, "count" : 5 }
    { "_id" : "Pop", "avgRank" : 314.51666666666665, "count" : 240 }
    { "_id" : "R&B", "avgRank" : 304.28387096774196, "count" : 155 }
    { "_id" : "Rock", "avgRank" : 297.1666666666667, "count" : 108 }
    { "_id" : "Country", "avgRank" : 285.95652173913044, "count" : 23 }
    { "_id" : "Hip-Hop/Rap", "avgRank" : 270.71794871794873, "count" : 39 }
    { "_id" : "Dance/Electronic", "avgRank" : 228.85185185185185, "count" : 27 }
    { "_id" : "Latin", "avgRank" : 137.66666666666666, "count" : 3 }

</li><br>
</ol>

<h4>Exporting a Collection</h4>

Just as you can import a .csv or .json file, you can also export a database collection to either file formats. Like the mongodb import command, the export must be run directly from the system command line. The simplest syntax for executing the program is:

    mongoexport --db <database> --collection <collectionName> --fields* <fieldList> --type* csv --out <filePath/fileName>
    
The fields parameter is optional. If its ommitted, all fields are included by default. The type field is only required if you're exporting to a 'csv file. This is what the export might look like if we were exporting the 'songs' collection from the 'top600' database to a .json file.

    mongoexport --db top600 --collection songs --out top600Songs.json
    
In this notebook our primary focus is on using Python in combination with JSON and MongoDB.  If you're using MongoDB primarily as 'storage' facilty for Python dictionaries (or dataframes for that matter), rather than having your files scattered through various directories, then you could simply use json (or csv) import and export to traffic your datasets between MongoDB and Python. On the other hand, if your interested in employing the analytical capabilities of Mongo in combination with Python, then you should focus on the module or package known as PyMongo which is what we're doing in the next section of the notebook.

<h3>PyMongo: Accessing MongoDB from Python</h3>

In Python when you read and write data to the file system, you first need to "open" and subsequently "close" a <i>connection</i> between your program and the operating system in order to perform these and other file  operations. The same thing is true with MongoDB. In order to perform any action on a MongoDB database or collection, you first need to establish a <i>database connection</i> between your Python program and the MongoDB server. This isn't unique to MongoDB. Virtually every external database requires a simliar sort of database connection in order to receive commands and return answers.

Database connections are usually enabled with <i>database drivers</i>. "A database driver is a computer program that implements a protocol for a database connection. The driver works like an adaptor which connects a generic interface to a specific database vendor implementation." In Python the <i>PyMongo</i> library is a native driver that is provided by MongoDB (the company), enabling Python programs to access MongoDB server.

In Python, PyMongo can be used to set up a connection with MongoDB is few ways. If you happen to be running MongoDB on the same PC as your Python program, one of the simplest is to create an instance of a PyMongo 'MongoClient' which in turn establishes a connection. The code immediately below illustrates the process. To demonstrate that the connection has been established, the code also lists the databases that are available and lists the collections for the 'top600' database which is the one we've been using (above) to illustrate various MongoDB operations. Additionally, I've provided two screen shots representing the log of the MongoDB server activities. The one on the left shows what occurs right after the server has been started (waiting for a connection). The one on the right is the screen log entries after the connection has been made.

In [25]:
import pymongo

# here the localhost is 127.0.0.0 and the port is 27017
# which are the designations we saw when we used the MongoDB shell was started

# the MongoClient creates an object which opens a connection (pool)
dbClient = pymongo.MongoClient("mongodb://localhost:27017/")

print("PyMongo 'show dbs'")
dbsList = dbClient.list_database_names()
print(dbsList)
print("")

# select database
currDBS = dbClient["top600"]

# list of collections in selected DBS
collList = currDBS.list_collection_names()

print("PyMongo 'show collections'")
print(collList)


PyMongo 'show dbs'
['admin', 'census', 'config', 'local', 'songs', 'testdb', 'top600']

PyMongo 'show collections'
['avgRanks', 'songArtists', 'songs', 'reviews']


![MongoDB Server Log](mongoserver2.jpg "Screen shots of MongoDB Server Log")

Now that we've established the connection and selected a particular database (i.e. currDB = dbClient["top600"]), we can select one or more of the collections and issue various MongoDB commands or operations in much the same way that we did with the MongoDB shell. In fact I'm going to basically mirror some of the 'find' and 'aggregate' operations that we executed with the shell. Again, this barely skims the surface of what can be done using PyMongo.  For more details, see one or more of the following references:

<ul>
    <li><a href="https://www.w3schools.com/python/python_mongodb_getstarted.asp">W3Schools Getting Started</a></li>
    <li><a href="https://api.mongodb.com/python/current/tutorial.html">MongoDB API Tutorial</a></li>
    <li><a href="http://zetcode.com/python/pymongo/">Zetcode PyMong Tutorial</a></li>
</ul>


<h5>Using 'find' in PyMongo</h5>

As a first step we going to query the 'songs' collection in the 'top600' database using the 'find' operation.  There are 600 documents in this collection. To simplify the discussion in this first example:

    songsCursor = songsCollection.find().sort([('_id', 1)]).limit(5))

I've sorted (ascending) and 'limit'ed the collection returned by 'find' to 5 documents.  We'll go back to the entire 600 shortly. The purpose of the first example is to explain primarily the concept of a database cursor.

<h5>MongoDB Cursor Explained</h5>

In MongoDB when a <i>db.collection.find()</i> function is used to search for documents in a collection, instead of actually returning a list of documents, the 'find' returns a pointer to the collection of documents.  That pointer is called a <i>cursor</i>. The cursor can then be used to iterate thru the collection specified in the 'find'. With a large collection of documents, this reduces the chance that you'll accidentally print or display the entire collection, although I eliminated this possibility by 'limit'ing the 'find' to 5 documents.

There are a number of ways to advance or iterate the cursor or pointer. One is to use the 'next()' method. This was done below with the 'songsCursor'. On the first call to 'next', the pointer is advanced to the first document which is the one shown. On the second call, the cursor is advanced to the second document which is then displayed. The next command can only go forward which means that the cursor can only access the remaining documents (in this case documents with the \_id's 3, 4 and 5. The only way to go back is to use the 'rewind()' method which moves the cursor to the beginning

One way to avoid some of the limitations associated with cursors is to covert the cursor to a list which was done with the Python 'list' function. In this way you can traverse the list in anyway you see fit. Of course, there is overhead, especially with larger collections.

In [26]:
# select songArtists collection from 'top and then find all docs in collection

import pymongo
dbClient = pymongo.MongoClient("mongodb://localhost:27017/")    
currDBS = dbClient["top600"]

print("1--Select collection of 'songs' in the currDBS to the variable songsCollection:")

songsCollection = currDBS.songs

print("  Python type of 'songsCollection':", type(songsCollection))
print("  Number of Documents in songsCollection:", songsCollection.count_documents({}))
print("")
print("2--'find' all documents in current collection and assign results to songsCursor:")
print("  the results are sorted and limited to first 5 documents for illustrative purposes")

songsCursor = songsCollection.find().sort([('_id', 1)]).limit(5)

print("  Python type of 'songsCursor:'", type(songsCursor))
print("")

#iterate thru songsCursor
print("6--Iterating the cursor with the 'next' method, i.e. songsCursor.next()")
print("")
print("--Next document: Displays data for first song")

print(songsCursor.next())

print("")
print("--Next document: Displays data for second song")

print(songsCursor.next())

print("")
print("--What remains is documents 3, 4, and 5: ")
print("")

for cur in songsCursor:
    print(cur)
    
print("")
print("7--rewind cursor with songsCursor.rewind() - resets cursor or pointer to first document")
print("  so we're back to 5 documents:")

songsCursor.rewind()

print("")
      
for cur in songsCursor:
    print(cur)
    
print("")

songsCursor.rewind()

print("8--Create a list from the songsCursor -- songsList = list(songsCursor).")
print(" Eliminates barriers to traversing the collection")
songsList = list(songsCursor)
print("")
print("song in position 5 of list:")
print(songsList[4])
print("")
print("song in position 3 of list:")
print(songsList[2])

1--Select collection of 'songs' in the currDBS to the variable songsCollection:
  Python type of 'songsCollection': <class 'pymongo.collection.Collection'>
  Number of Documents in songsCollection: 600

2--'find' all documents in current collection and assign results to songsCursor:
  the results are sorted and limited to first 5 documents for illustrative purposes
  Python type of 'songsCursor:' <class 'pymongo.cursor.Cursor'>

6--Iterating the cursor with the 'next' method, i.e. songsCursor.next()

--Next document: Displays data for first song
{'_id': 1, 'Rank': 1, 'Artist': 'Chubby Checker', 'SongTitle': 'THE TWIST', 'Gender': 'Male', 'Genre': 'R&B', 'Decade': '60s'}

--Next document: Displays data for second song
{'_id': 2, 'Rank': 2, 'Artist': 'Santana Featuring Rob Thomas', 'SongTitle': 'SMOOTH', 'Gender': 'Duo/Group', 'Genre': 'Rock', 'Decade': '90s'}

--What remains is documents 3, 4, and 5: 

{'_id': 3, 'Rank': 3, 'Artist': 'Bobby Darin', 'SongTitle': 'MACK THE KNIFE', 'Gender


{'_id': 1, 'Rank': 1, 'Artist': 'Chubby Checker', 'SongTitle': 'THE TWIST', 'Gender': 'Male', 'Genre': 'R&B', 'Decade': '60s'}
{'_id': 2, 'Rank': 2, 'Artist': 'Santana Featuring Rob Thomas', 'SongTitle': 'SMOOTH', 'Gender': 'Duo/Group', 'Genre': 'Rock', 'Decade': '90s'}
{'_id': 3, 'Rank': 3, 'Artist': 'Bobby Darin', 'SongTitle': 'MACK THE KNIFE', 'Gender': 'Male', 'Genre': 'Pop', 'Decade': '50s'}
{'_id': 4, 'Rank': 4, 'Artist': 'Mark Ronson Featuring Bruno Mars', 'SongTitle': 'UPTOWN FUNK!', 'Gender': 'Male', 'Genre': 'Pop', 'Decade': '10s'}
{'_id': 5, 'Rank': 5, 'Artist': 'LeAnn Rimes', 'SongTitle': 'HOW DO I LIVE', 'Gender': 'Female', 'Genre': 'Country', 'Decade': '90s'}



8--Create a list from the songsCursor -- songsList = list(songsCursor).
 Eliminates barriers to traversing the collection

song in position 5 of list:
{'_id': 5, 'Rank': 5, 'Artist': 'LeAnn Rimes', 'SongTitle': 'HOW DO I LIVE', 'Gender': 'Female', 'Genre': 'Country', 'Decade': '90s'}

song in position 3 of list:
{'_id': 3, 'Rank': 3, 'Artist': 'Bobby Darin', 'SongTitle': 'MACK THE KNIFE', 'Gender': 'Male', 'Genre': 'Pop', 'Decade': '50s'}


This second example illustrates the 'find' function with a more complex set of operators.  Here, we're using the $text operator to search for the words LOVE or LOVES appearing in the SongTitle. Technically, because a text index was created for the documents in the 'songs' collection, if either word appears as a substring in any field that contains strings, e.g. the name field. While they could appear in any field as long as its values are strings, they don't 

In [27]:
# Example more complex 'find' function

import pymongo
dbClient = pymongo.MongoClient("mongodb://localhost:27017/")    
currDBS = dbClient["top600"]
songsCollection = currDBS.songs

print("Find Songs with LOVE or LOVES in the title. Results saved to 'lovesSongsCursor'")
print('songsCollection.find({"$text":{"$search": "LOVE LOVES", "$caseSensitive": True}}).sort([("_id", 1)])')
loveSongsCursor = songsCollection.find({"$text":{"$search": "LOVE LOVES", "$caseSensitive": True}}).sort([("_id", 1)])
print("")

print("First love Song based on loveSongsCursor:")
print("")
print(loveSongsCursor.next())
print("")

print("rewind longSongsCursor to create a Python 'list' of love songs")
loveSongsCursor.rewind()
print("Create a list of songs based from loveSongsCursor from find. ")
print("")
loveSongsList = list(loveSongsCursor)

print("Number of Songs with LOVE or LOVES in title:")
print(len(loveSongsList))
print("")

print('First 5 songs based on loveSongsList:')
for song in loveSongsList[0:5]:
    print(song)


Find Songs with LOVE or LOVES in the title. Results saved to 'lovesSongsCursor'
songsCollection.find({"$text":{"$search": "LOVE LOVES", "$caseSensitive": True}}).sort([("_id", 1)])

First love Song based on loveSongsCursor:

{'_id': 18, 'Rank': 18, 'Artist': 'Diana Ross & Lionel Richie', 'SongTitle': 'ENDLESS LOVE', 'Gender': 'Male/Female', 'Genre': 'R&B', 'Decade': '80s'}

rewind longSongsCursor to create a Python 'list' of love songs


Create a list of songs based from loveSongsCursor from find. 

Number of Songs with LOVE or LOVES in title:
52

First 5 songs based on loveSongsList:
{'_id': 18, 'Rank': 18, 'Artist': 'Diana Ross & Lionel Richie', 'SongTitle': 'ENDLESS LOVE', 'Gender': 'Male/Female', 'Genre': 'R&B', 'Decade': '80s'}
{'_id': 22, 'Rank': 22, 'Artist': 'Boyz II Men', 'SongTitle': "I'LL MAKE LOVE TO YOU", 'Gender': 'Duo/Group', 'Genre': 'R&B', 'Decade': '90s'}
{'_id': 25, 'Rank': 25, 'Artist': 'Bee Gees', 'SongTitle': 'HOW DEEP IS YOUR LOVE', 'Gender': 'Duo/Group', 'Genre': 'Pop', 'Decade': '70s'}
{'_id': 27, 'Rank': 27, 'Artist': 'Rihanna Featuring Calvin Harris', 'SongTitle': 'WE FOUND LOVE', 'Gender': 'Female', 'Genre': 'Pop', 'Decade': '10s'}
{'_id': 40, 'Rank': 40, 'Artist': 'Wings', 'SongTitle': 'SILLY LOVE SONGS', 'Gender': 'Duo/Group', 'Genre': 'Rock', 'Decade': '70s'}


<h5>Using Aggregate in PyMongo</h5>

The final two examples are replays of the discussion about Aggregation Pipelines in MongoDB, except this time we're performing the same analysis with PyMongo.

In [28]:
import pymongo
dbClient = pymongo.MongoClient("mongodb://localhost:27017/")    
currDBS = dbClient["top600"]
songsCollection = currDBS.songs

# use aggregate to determine counts for number of songs by Decade
pipeLine = [{"$group":{"_id":"$Decade", "count":{"$sum":1}}}]
songCntsDecade = list(songsCollection.aggregate(pipeLine))
print('Number of Songs by Decade using $group aggregate stage')
for cnt in songCntsDecade:
    print(cnt)
print("")

# use aggregate to determine average rank by Genre
print('Average Rank by Genre using aggregate stages $project, $group and $sort')
pipeLine = [{"$project": {"Rank":1, "Genre":1, "_id":1}}, 
            {"$group":{"_id":"$Genre", "avgRank":{"$avg":"$Rank"},"count":{"$sum":1}}},
            {"$sort":{"avgRank":-1}}]
songCntsGenre = list(songsCollection.aggregate(pipeLine))

for cnt in songCntsGenre:
    print(cnt)

Number of Songs by Decade using $group aggregate stage
{'_id': '70s', 'count': 114}
{'_id': '80s', 'count': 133}
{'_id': '10s', 'count': 71}
{'_id': '00s', 'count': 83}
{'_id': '90s', 'count': 102}
{'_id': '50s', 'count': 21}
{'_id': '60s', 'count': 76}

Average Rank by Genre using aggregate stages $project, $group and $sort
{'_id': 'Jazz', 'avgRank': 366.2, 'count': 5}
{'_id': 'Pop', 'avgRank': 314.51666666666665, 'count': 240}
{'_id': 'R&B', 'avgRank': 304.28387096774196, 'count': 155}
{'_id': 'Rock', 'avgRank': 297.1666666666667, 'count': 108}
{'_id': 'Country', 'avgRank': 285.95652173913044, 'count': 23}
{'_id': 'Hip-Hop/Rap', 'avgRank': 270.71794871794873, 'count': 39}
{'_id': 'Dance/Electronic', 'avgRank': 228.85185185185185, 'count': 27}
{'_id': 'Latin', 'avgRank': 137.66666666666666, 'count': 3}


<h4>Summary</h4>

This notebook briefly introduced the combined use of Python, JSON, MongoDB and PyMongo to analyze 'semi-structured' data. In the case of MongoDB and PyMongo, the focus has been primarily on the 'R' or 'read' in the 'CRUD' of databases. With respect to the 'C' or creation of databases, the primary method used here revolved around 'mongoimport' and 'mongoexport'. Finally, we haven't really touched in any meaningful way on 'U' or 'D', update and delete, respectively.  For the 'CUD' side of database life, I've listed other detailed, free sources that you can refer to. If you're willing to shell out a few dollars (or whatever currency), you look at a couple older books and a more recent one including

<ul>
<li><a href="https://www.packtpub.com/big-data-and-business-intelligence/instant-mongodb-instant">Instant MongoDB (2013) by Amol Nayak</a></li>
<li><a href="http://img105.job1001.com/upload/adminnew/2015-04-07/1428393873-MHKX3LN.pdf">MongoDB and Python: Patterns and processes for the popular document-oriented database (2011) by Niall O'Higgins</a> Note: there is a free pdf</li>
<li><a href="https://www.apress.com/us/book/9781484235966">Fundamentals of Data Science for Python and MongoDB (2018) by David Paper</a>
 </ul></li>